In [16]:
from torch_geometric.data import Data
import pandas as pd
import json
import numpy as np
from sklearn import preprocessing
import torch
import sys
import os
import pickle

In [18]:
import pathlib
src_dir = os.path.dirname(str(pathlib.Path().resolve()))
data_dir = src_dir+"/data/Mimic3/"
print(data_dir)
edge_list = pickle.load(open(data_dir+"/edge_list.pkl","rb"))
admissions = pd.read_pickle(data_dir+"/processed_admissions.pkl")
print(len(edge_list),admissions.shape)

/home/ctgnn/ctgnn/src/data/Mimic3/
111175 (11840, 9)


In [19]:
admissions.head()

,HADM_ID,SUBJECT_ID,MARITAL_STATUS,HOSPITAL_EXPIRE_FLAG,DIAGNOSIS,GENDER,PROCED_ICD9_CODES,DRUGS,LAB_ITEMIDS
0,165315,22,MARRIED,0,BENZODIAZEPINE OVERDOSE,F,[96],[],"[51221, 51279, 50931, 50971]"
1,129635,25,MARRIED,0,ACUTE CORONARY SYNDROME,M,"[88, 37, 99, 36]","[Heparin Sodium, D5 1/2NS, Dextrose 5%, D5W, N...","[50902, 50908, 50911, 50912, 50931, 51003, 510..."
2,188670,37,MARRIED,0,CONGESTIVE HEART FAILURE,M,"[45, 99]","[Eptifibatide, SW, Clopidogrel Bisulfate, Digo...","[50910, 50911, 50931, 51003, 51006, 51221, 512..."
3,101651,357,MARRIED,0,PNEUMONIA,M,"[96, 38]","[Lactulose, LeVETiracetam, 1/2 NS, LeVETiracet...","[50863, 50882, 50902, 50931, 50963, 50983, 509..."
4,117876,357,MARRIED,0,ALTERED MENTAL STATUS,M,"[96, 14, 34, 38]","[Linezolid, Potassium Chloride, Vial, Pantopra...","[51250, 51265, 51274, 51277, 51279, 51301, 509..."


In [20]:
def get_item_to_id_dict(lsts):
    dict_ = {}
    for j in lsts:
        if j != '' and j not in dict_:
            dict_[j] = len(dict_)
    return dict_
def get_items_to_id_dict(lsts):
    dict_ = {}
    for i in lsts:
        for j in i:
            if j not in dict_:
                dict_[j] = len(dict_)
    return dict_

In [22]:
node_to_id = get_item_to_id_dict(admissions['HADM_ID'])
gender_to_id = get_item_to_id_dict(admissions['GENDER'])
proced_to_id = get_items_to_id_dict(admissions['PROCED_ICD9_CODES'])
drugs_to_id = get_items_to_id_dict(admissions['DRUGS'])
lab_items_to_id = get_items_to_id_dict(admissions['LAB_ITEMIDS'])

In [30]:
final_edge_list = []
for edge in edge_list:
    node1 = node_to_id[edge[0]]
    node2 = node_to_id[edge[1]]
    final_edge_list.append((node1,node2))
    final_edge_list.append((node2,node1))
print(len(final_edge_list))

print(len(final_edge_list))
final_edge_list = list(set(final_edge_list))
print(len(final_edge_list))
assert np.max(final_edge_list) + 1 == len(node_to_id)
final_edge_list = np.transpose(final_edge_list)
edge_index = torch.tensor(final_edge_list,dtype=torch.long)
print(edge_index.shape)

222350


In [51]:
Y = [0]*len(node_to_id)
node_feats = [0]*len(node_to_id)
for node,HOSPITAL_EXPIRE_FLAG,GENDER,PROCED_ICD9_CODES,DRUGS,LAB_ITEMIDS in admissions[['HADM_ID','HOSPITAL_EXPIRE_FLAG','GENDER','PROCED_ICD9_CODES','DRUGS','LAB_ITEMIDS']].values:   
    index = node_to_id[node]
    y_label = int(HOSPITAL_EXPIRE_FLAG)
    gender_vector = np.zeros(2)
    proced_vector = np.zeros(len(proced_to_id))
    drug_vector = np.zeros(len(drugs_to_id))
    lab_item_vector = np.zeros(len(lab_items_to_id))
    if GENDER in gender_to_id:
        gender_vector[gender_to_id[GENDER]] = 1
    for item in PROCED_ICD9_CODES:
        proced_vector[proced_to_id[item]]= 1
    for item in DRUGS:
        drug_vector[drugs_to_id[item]]= 1
    for item in LAB_ITEMIDS:
        lab_item_vector[lab_items_to_id[item]]= 1
    
    feat_vector = np.concatenate((gender_vector,proced_vector,drug_vector,lab_item_vector))
    #print(feat_vector.shape)
    #print(feat_vector.sum())
    #print(index)
    Y[index] = y_label
    node_feats[index] = feat_vector
Y = np.array(Y)
node_feats = np.array(node_feats)
print(len(Y),node_feats.shape)
print(sum(node_feats.sum(axis=1)==0))

node_feats = torch.tensor(node_feats,dtype=torch.bool) ## boolean features present or absent
print(node_feats.shape)

11840 (11840, 1186)


In [67]:

data = Data(
        x=node_feats,
        edge_index=edge_index,
        y=torch.tensor(Y,dtype=torch.long),
        train_mask = torch.ones(len(Y),dtype=torch.bool), ### placeholder for masks
        val_mask = torch.ones(len(Y),dtype=torch.bool), ### placeholder for masks
        test_mask = torch.ones(len(Y),dtype=torch.bool), ### placeholder for masks
        num_nodes = node_feats.shape[0],
        num_features = node_feats.shape[1]

)

In [68]:
data

Data(x=[11840, 1186], edge_index=[2, 222350], y=[11840], train_mask=[11840], val_mask=[11840], test_mask=[11840], num_nodes=11840, num_features=1186)

torch.Size([2, 222350])


In [32]:

    edges = pd.read_csv(edge_file)
    features = json.load(open(feature_file))
    targets = pd.read_csv(target_file)

    edge_list = np.array(create_edge_list(edges))
    node_targets = extract_target(targets,data_type)
    assert np.max(edge_list) + 1 == len(node_targets)
    Y = []
    for node_id in range(0, len(node_targets)):
        Y.append(node_targets[node_id])
    feature_matrix = extract_feature_matrix(features,node_targets)


    edge_list = np.transpose(edge_list)
    #print(edge_list.shape)
    node_feat = torch.tensor(feature_matrix,dtype=torch.bool) ## boolean features present or absent

    data = Data(
            x=node_feat,
            edge_index=edge_index,
            y=torch.tensor(Y,dtype=torch.long),
            train_mask = torch.ones(len(Y),dtype=torch.bool), ### placeholder for masks
            val_mask = torch.ones(len(Y),dtype=torch.bool), ### placeholder for masks
            test_mask = torch.ones(len(Y),dtype=torch.bool), ### placeholder for masks
            num_nodes = node_feat.shape[0],
            num_features = node_feat.shape[1]

    )
    return data

NameError: name 'edge_file' is not defined